In [4]:
import pandas as pd
import numpy as np

import os


# Task 1

In [5]:
Project_path = os.getcwd()
print (Project_path)


C:\Users\nahani\Dropbox\Public-cQuant.io


In [6]:
Data_folder = Project_path+'\\Data'
Output_folder = Project_path+'\\Output'

In [10]:
contracts =  pd.read_csv(os.path.join(Data_folder+'\\contracts', 'Contracts.csv'))

fuelPrices_GDA_TETSTX = pd.read_csv(os.path.join(Data_folder+'\\fuelPrices', 'GDA_TETSTX.csv'))
fuelPrices_Henry_Hub = pd.read_csv(os.path.join(Data_folder+'\\fuelPrices', 'Henry Hub.csv'))

plantParameters = pd.read_csv(os.path.join(Data_folder+'\\plantParameters', 'Plant_Parameters.csv'))

powerPrices_2016 = pd.read_csv(os.path.join(Data_folder+'\\powerPrices', 'ERCOT_DA_Prices_2016.csv'))
powerPrices_2017 = pd.read_csv(os.path.join(Data_folder+'\\powerPrices', 'ERCOT_DA_Prices_2017.csv'))
powerPrices_2018 = pd.read_csv(os.path.join(Data_folder+'\\powerPrices', 'ERCOT_DA_Prices_2018.csv'))
powerPrices_2019 = pd.read_csv(os.path.join(Data_folder+'\\powerPrices', 'ERCOT_DA_Prices_2019.csv'))

In [11]:
fuelPrices = pd.concat([fuelPrices_GDA_TETSTX , fuelPrices_Henry_Hub])

In [215]:
powerPrices = pd.concat([powerPrices_2016, powerPrices_2017, powerPrices_2018, powerPrices_2019])

# Task 2

In [218]:
powerPrices['Y-M'] = pd.to_datetime(powerPrices['Date']).dt.strftime('%m/%Y')


In [25]:
grouped = powerPrices.groupby(['SettlementPoint','Y-M']).agg({'Price': ['mean', 'min', 'max', 'std']})

In [26]:
grouped

Price                          
                              mean   min      max        std
SettlementPoint Y-M                                         
HB_BUSAVG       01/2016  19.209301  5.12    54.69   5.485210
                01/2017  23.252782  1.94    96.28   7.471613
                01/2018  37.869677  5.83  1660.04  87.563613
                01/2019  24.592567  5.72    75.28   8.130355
                02/2016  15.288247  1.79    50.98   5.777172
...                            ...   ...      ...        ...
LZ_WEST         11/2019  49.340847  3.60   286.91  31.876126
                12/2016  25.165820  0.37   139.73  11.654195
                12/2017  21.872258 -1.49    64.70   7.489398
                12/2018  34.980847  3.05   131.59  15.254886
                12/2019  39.502594  7.32   195.71  27.975862

[681 rows x 4 columns]

# Task 3

I'll get back to it

# Task 4

In [32]:
grouped.columns.to_list()
# grouped_df = pd.DataFrame(grouped, columns=[''])


[('Price', 'mean'), ('Price', 'min'), ('Price', 'max'), ('Price', 'std')]

In [41]:
grouped.columns = ['Mean', 'Min', 'Max', 'SD']

In [44]:
grouped.reset_index(inplace=True)

In [46]:
grouped['Year'] = grouped['Y-M'].apply(lambda x: x.split('/')[1])
grouped['Month'] = grouped['Y-M'].apply(lambda x: x.split('/')[0])

In [50]:
grouped.drop(['Y-M'],  axis=1, inplace=True)

In [53]:
grouped.to_csv(os.path.join(Output_folder, 'MonthlyPowerPriceStatistics.csv'))

# Task 5

In [76]:
contracts

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [77]:
contracts_hourly = contracts[contracts['Granularity'] == 'Hourly']
contracts_daily = contracts[contracts['Granularity'] == 'Daily']

In [78]:
contracts_daily

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
0,S1,Swap,2017-01-01,2017-03-31,20000,Daily,3.0,NaN,Henry Hub
2,O1,European option,2017-06-01,2017-07-15,10000,Daily,2.9,0.1,GDA_TETSTX


In [79]:
contracts_hourly

,ContractName,DealType,StartDate,EndDate,Volume,Granularity,StrikePrice,Premium,PriceName
1,S2,Swap,2017-01-01,2018-12-31,1000,Hourly,21.0,NaN,HB_NORTH
3,O2,European option,2018-10-01,2019-09-30,1000,Hourly,31.5,3.0,HB_HOUSTON


In [80]:
contracts_daily['Date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(contracts_daily['StartDate']),
                  pd.to_datetime(contracts_daily['EndDate']))]

contracts_daily = contracts_daily.explode('Date').drop(['StartDate', 'EndDate'], axis=1)

<ipython-input-80-99b764c7e2ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_daily['Date'] = [pd.date_range(s, e, freq='d') for s, e in


In [81]:
contracts_daily

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-01
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-02
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-03
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-04
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-05
...,...,...,...,...,...,...,...,...
2,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-11
2,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-12
2,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-13
2,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-14


In [82]:
contracts_hourly['Date'] = [pd.date_range(s, e, freq='1H') for s, e in
              zip(pd.to_datetime(contracts_hourly['StartDate']),
                  pd.to_datetime(contracts_hourly['EndDate']))]

contracts_hourly = contracts_hourly.explode('Date').drop(['StartDate', 'EndDate'], axis=1)

<ipython-input-82-f953e24bd43c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_hourly['Date'] = [pd.date_range(s, e, freq='1H') for s, e in


In [83]:
contracts_hourly

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 00:00:00
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 01:00:00
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 02:00:00
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 03:00:00
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 04:00:00
...,...,...,...,...,...,...,...,...
3,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 20:00:00
3,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 21:00:00
3,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 22:00:00
3,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 23:00:00


# Task 6

In [99]:
#fuel prices for daily contracts
contracts_daily['Date'] = pd.to_datetime(contracts_daily['Date'])
fuelPrices['Date'] = pd.to_datetime(fuelPrices['Date'])

contracts_daily_fuelPrices = pd.merge(contracts_daily, fuelPrices,  
                                      how='left', 
                                      left_on=['PriceName','Date'], 
                                      right_on = ['Variable','Date'])

In [100]:
contracts_daily_fuelPrices

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,Variable,Price
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-01,Henry Hub,3.650
1,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-02,Henry Hub,3.650
2,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-03,Henry Hub,3.650
3,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-04,Henry Hub,3.390
4,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-05,Henry Hub,3.370
...,...,...,...,...,...,...,...,...,...,...
130,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-11,GDA_TETSTX,2.910
131,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-12,GDA_TETSTX,2.910
132,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-13,GDA_TETSTX,2.915
133,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-14,GDA_TETSTX,2.865


In [97]:
# power prices for hourly contracts

contracts_hourly['Date'] = pd.to_datetime(contracts_hourly['Date'])
powerPrices['Date'] = pd.to_datetime(powerPrices['Date'])

contracts_hourly_powerPrices = pd.merge(contracts_hourly, powerPrices,  
                                      how='left', 
                                      left_on=['PriceName','Date'], 
                                      right_on = ['SettlementPoint','Date'])

In [98]:
contracts_hourly_powerPrices

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,SettlementPoint,Price,Y-M
0,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 00:00:00,HB_NORTH,23.23,01/2017
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 01:00:00,HB_NORTH,21.82,01/2017
2,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 02:00:00,HB_NORTH,20.16,01/2017
3,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 03:00:00,HB_NORTH,20.00,01/2017
4,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 04:00:00,HB_NORTH,20.24,01/2017
...,...,...,...,...,...,...,...,...,...,...,...
26229,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 20:00:00,HB_HOUSTON,23.71,09/2019
26230,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 21:00:00,HB_HOUSTON,20.95,09/2019
26231,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 22:00:00,HB_HOUSTON,18.56,09/2019
26232,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 23:00:00,HB_HOUSTON,17.89,09/2019


# Task 7

In [102]:
contracts_hourly_powerPrices.fillna(0)

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,SettlementPoint,Price,Y-M
0,S2,Swap,1000,Hourly,21.0,0.0,HB_NORTH,2017-01-01 00:00:00,HB_NORTH,23.23,01/2017
1,S2,Swap,1000,Hourly,21.0,0.0,HB_NORTH,2017-01-01 01:00:00,HB_NORTH,21.82,01/2017
2,S2,Swap,1000,Hourly,21.0,0.0,HB_NORTH,2017-01-01 02:00:00,HB_NORTH,20.16,01/2017
3,S2,Swap,1000,Hourly,21.0,0.0,HB_NORTH,2017-01-01 03:00:00,HB_NORTH,20.00,01/2017
4,S2,Swap,1000,Hourly,21.0,0.0,HB_NORTH,2017-01-01 04:00:00,HB_NORTH,20.24,01/2017
...,...,...,...,...,...,...,...,...,...,...,...
26229,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 20:00:00,HB_HOUSTON,23.71,09/2019
26230,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 21:00:00,HB_HOUSTON,20.95,09/2019
26231,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 22:00:00,HB_HOUSTON,18.56,09/2019
26232,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 23:00:00,HB_HOUSTON,17.89,09/2019


In [103]:
contracts_daily_fuelPrices.fillna(0)

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,Variable,Price
0,S1,Swap,20000,Daily,3.0,0.0,Henry Hub,2017-01-01,Henry Hub,3.650
1,S1,Swap,20000,Daily,3.0,0.0,Henry Hub,2017-01-02,Henry Hub,3.650
2,S1,Swap,20000,Daily,3.0,0.0,Henry Hub,2017-01-03,Henry Hub,3.650
3,S1,Swap,20000,Daily,3.0,0.0,Henry Hub,2017-01-04,Henry Hub,3.390
4,S1,Swap,20000,Daily,3.0,0.0,Henry Hub,2017-01-05,Henry Hub,3.370
...,...,...,...,...,...,...,...,...,...,...
130,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-11,GDA_TETSTX,2.910
131,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-12,GDA_TETSTX,2.910
132,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-13,GDA_TETSTX,2.915
133,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-14,GDA_TETSTX,2.865


In [114]:
contracts_daily_fuelPrices['max_price_strike'] = np.where(contracts_daily_fuelPrices['Price']-contracts_daily_fuelPrices['StrikePrice']>0,
                                                         contracts_daily_fuelPrices['Price']-contracts_daily_fuelPrices['StrikePrice'],
                                                         0)

In [117]:
# • Swaps: (AssetPrice – StrikePrice) x Volume
# • Options: [MAX(AssetPrice – StrikePrice, 0) - Premium] x Volume
contracts_daily_fuelPrices['payoffs'] = np.where(
 contracts_daily_fuelPrices['DealType'] == 'Swap',
 (contracts_daily_fuelPrices['Price']-contracts_daily_fuelPrices['StrikePrice'])*contracts_daily_fuelPrices['Volume'],
 (contracts_daily_fuelPrices['max_price_strike']-contracts_daily_fuelPrices['Premium'])*contracts_daily_fuelPrices['Volume']
)

In [118]:
contracts_daily_fuelPrices

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,Variable,Price,payoffs,max_price_strike
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-01,Henry Hub,3.650,13000.0,0.650
1,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-02,Henry Hub,3.650,13000.0,0.650
2,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-03,Henry Hub,3.650,13000.0,0.650
3,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-04,Henry Hub,3.390,7800.0,0.390
4,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-05,Henry Hub,3.370,7400.0,0.370
...,...,...,...,...,...,...,...,...,...,...,...,...
130,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-11,GDA_TETSTX,2.910,-900.0,0.010
131,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-12,GDA_TETSTX,2.910,-900.0,0.010
132,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-13,GDA_TETSTX,2.915,-850.0,0.015
133,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-14,GDA_TETSTX,2.865,-1000.0,0.000


In [119]:
contracts_hourly_powerPrices['max_price_strike'] = np.where(contracts_hourly_powerPrices['Price']-contracts_hourly_powerPrices['StrikePrice']>0,
         contracts_hourly_powerPrices['Price']-contracts_hourly_powerPrices['StrikePrice'],
                                                         0)

In [120]:
contracts_hourly_powerPrices['payoffs'] = np.where(
 contracts_hourly_powerPrices['DealType'] == 'Swap',
 (contracts_hourly_powerPrices['Price']-contracts_hourly_powerPrices['StrikePrice'])*contracts_hourly_powerPrices['Volume'],
 (contracts_hourly_powerPrices['max_price_strike']-contracts_hourly_powerPrices['Premium'])*contracts_hourly_powerPrices['Volume']
)

In [121]:
contracts_hourly_powerPrices

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,SettlementPoint,Price,Y-M,max_price_strike,payoffs
0,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 00:00:00,HB_NORTH,23.23,01/2017,2.23,2230.0
1,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 01:00:00,HB_NORTH,21.82,01/2017,0.82,820.0
2,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 02:00:00,HB_NORTH,20.16,01/2017,0.00,-840.0
3,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 03:00:00,HB_NORTH,20.00,01/2017,0.00,-1000.0
4,S2,Swap,1000,Hourly,21.0,NaN,HB_NORTH,2017-01-01 04:00:00,HB_NORTH,20.24,01/2017,0.00,-760.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26229,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 20:00:00,HB_HOUSTON,23.71,09/2019,0.00,-3000.0
26230,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 21:00:00,HB_HOUSTON,20.95,09/2019,0.00,-3000.0
26231,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 22:00:00,HB_HOUSTON,18.56,09/2019,0.00,-3000.0
26232,O2,European option,1000,Hourly,31.5,3.0,HB_HOUSTON,2019-09-29 23:00:00,HB_HOUSTON,17.89,09/2019,0.00,-3000.0


# Task 8

In [123]:
contracts_daily_fuelPrices ['Y-M'] = pd.to_datetime(contracts_daily_fuelPrices['Date']).dt.strftime('%m/%Y')

In [124]:
contracts_daily_fuelPrices

,ContractName,DealType,Volume,Granularity,StrikePrice,Premium,PriceName,Date,Variable,Price,payoffs,max_price_strike,Y-M
0,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-01,Henry Hub,3.650,13000.0,0.650,01/2017
1,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-02,Henry Hub,3.650,13000.0,0.650,01/2017
2,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-03,Henry Hub,3.650,13000.0,0.650,01/2017
3,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-04,Henry Hub,3.390,7800.0,0.390,01/2017
4,S1,Swap,20000,Daily,3.0,NaN,Henry Hub,2017-01-05,Henry Hub,3.370,7400.0,0.370,01/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-11,GDA_TETSTX,2.910,-900.0,0.010,07/2017
131,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-12,GDA_TETSTX,2.910,-900.0,0.010,07/2017
132,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-13,GDA_TETSTX,2.915,-850.0,0.015,07/2017
133,O1,European option,10000,Daily,2.9,0.1,GDA_TETSTX,2017-07-14,GDA_TETSTX,2.865,-1000.0,0.000,07/2017


In [127]:
grouped_1 = contracts_daily_fuelPrices.groupby(['ContractName','Y-M']).agg({'payoffs': ['sum']})


In [128]:
grouped_1

payoffs
                           sum
ContractName Y-M              
O1           06/2017  -20100.0
             07/2017  -14300.0
S1           01/2017  198300.0
             02/2017  -97700.0
             03/2017  -97700.0

In [129]:
grouped_2 = contracts_hourly_powerPrices.groupby(['ContractName','Y-M']).agg({'payoffs': ['sum']})

In [130]:
grouped_2

payoffs
                             sum
ContractName Y-M                
O2           01/2019  -1320540.0
             02/2019  -1680820.0
             03/2019    678800.0
             04/2019   -730570.0
             05/2019  -1097720.0
             06/2019    773630.0
             07/2019   3355940.0
             08/2019  75340320.0
             09/2019  39174380.0
             10/2018    507430.0
             11/2018    598010.0
             12/2018     43140.0
S2           01/2017   1481940.0
             01/2018  12339910.0
             02/2017  -1042810.0
             02/2018    515440.0
             03/2017    169240.0
             03/2018   1194750.0
             04/2017    295600.0
             04/2018   1816500.0
             05/2017   1800420.0
             05/2018  10643770.0
             06/2017   3521210.0
             06/2018   6632560.0
             07/2017   7182810.0
             07/2018  43128530.0
             08/2017   4339550.0
             08/2018   7888670.0
             09/2017   1689200.0
             09/2018   4535740.0
             10/2017    301990.0
             10/2018   6721420.0
             11/2017    429270.0
             11/2018   7369120.0
             12/2017    -34970.0
             12/2018   6966000.0

In [140]:
output = pd.concat([grouped_1, grouped_2])

In [141]:
output

payoffs
                             sum
ContractName Y-M                
O1           06/2017    -20100.0
             07/2017    -14300.0
S1           01/2017    198300.0
             02/2017    -97700.0
             03/2017    -97700.0
O2           01/2019  -1320540.0
             02/2019  -1680820.0
             03/2019    678800.0
             04/2019   -730570.0
             05/2019  -1097720.0
             06/2019    773630.0
             07/2019   3355940.0
             08/2019  75340320.0
             09/2019  39174380.0
             10/2018    507430.0
             11/2018    598010.0
             12/2018     43140.0
S2           01/2017   1481940.0
             01/2018  12339910.0
             02/2017  -1042810.0
             02/2018    515440.0
             03/2017    169240.0
             03/2018   1194750.0
             04/2017    295600.0
             04/2018   1816500.0
             05/2017   1800420.0
             05/2018  10643770.0
             06/2017   3521210.0
             06/2018   6632560.0
             07/2017   7182810.0
             07/2018  43128530.0
             08/2017   4339550.0
             08/2018   7888670.0
             09/2017   1689200.0
             09/2018   4535740.0
             10/2017    301990.0
             10/2018   6721420.0
             11/2017    429270.0
             11/2018   7369120.0
             12/2017    -34970.0
             12/2018   6966000.0

# Task 9

In [142]:
output.reset_index(inplace=True, drop=False)
output.columns.to_list()


[('ContractName', ''), ('Y-M', ''), ('payoffs', 'sum')]

In [143]:
output.columns = ['ContractName', 'Y-M', 'TotalPayoff']

In [145]:
output['Year'] = output['Y-M'].apply(lambda x: x.split('/')[1])
output['Month'] = output['Y-M'].apply(lambda x: x.split('/')[0])
output.drop(['Y-M'],  axis=1, inplace=True)

In [147]:
output.to_csv(os.path.join(Output_folder, 'MonthlyContractPayoffs.csv'))

# Task 10

In [190]:
plantParameters

,PlantName,Year,Month,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost
0,Alpha,2017,1,HB_HOUSTON,Henry Hub,100,9.2,3,0.10,10000
1,Alpha,2017,2,HB_HOUSTON,Henry Hub,98,9.4,3,0.10,10000
2,Alpha,2017,3,HB_HOUSTON,Henry Hub,96,9.6,3,0.10,10000
3,Alpha,2017,4,HB_HOUSTON,Henry Hub,94,9.8,3,0.10,10000
4,Alpha,2017,5,HB_HOUSTON,Henry Hub,92,10.0,3,0.10,10000
5,Alpha,2017,6,HB_HOUSTON,Henry Hub,90,10.0,3,0.10,10000
6,Alpha,2017,7,HB_HOUSTON,Henry Hub,90,10.0,3,0.10,10000
7,Alpha,2017,8,HB_HOUSTON,Henry Hub,92,10.0,3,0.10,10000
8,Alpha,2017,9,HB_HOUSTON,Henry Hub,94,9.8,3,0.10,10000
9,Alpha,2017,10,HB_HOUSTON,Henry Hub,96,9.6,3,0.10,10000


In [185]:
fuelPrices['Year'] = pd.to_datetime(fuelPrices['Date']).dt.strftime('%Y')
fuelPrices['Month'] = pd.to_datetime(fuelPrices['Date']).dt.strftime('%m')

In [186]:
fuelPrices_17_18 = fuelPrices[(fuelPrices['Year'] =='2017') | (fuelPrices['Year'] == '2018')]

In [188]:
fuelPrices_17_18['Year']= fuelPrices_17_18['Year'].astype(str)
fuelPrices_17_18['Month']= fuelPrices_17_18['Month'].astype(str)

<ipython-input-188-6629c1d92b98>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuelPrices_17_18['Year']= fuelPrices_17_18['Year'].astype(str)
<ipython-input-188-6629c1d92b98>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuelPrices_17_18['Month']= fuelPrices_17_18['Month'].astype(str)


In [189]:
fuelPrices_17_18

,Variable,Date,Price,Year,Month
121,GDA_TETSTX,2018-12-31,3.080,2018,12
122,GDA_TETSTX,2018-12-30,3.080,2018,12
123,GDA_TETSTX,2018-12-29,3.080,2018,12
124,GDA_TETSTX,2018-12-28,3.080,2018,12
125,GDA_TETSTX,2018-12-27,2.915,2018,12
...,...,...,...,...,...
1091,Henry Hub,2018-12-27,3.215,2018,12
1092,Henry Hub,2018-12-28,3.065,2018,12
1093,Henry Hub,2018-12-29,3.065,2018,12
1094,Henry Hub,2018-12-30,3.065,2018,12


In [192]:
plantParameters['Year']= plantParameters['Year'].astype(str)
plantParameters['Month']= plantParameters['Month'].astype(str)


In [193]:
fuelPrices_17_18_plantParameters = pd.merge(fuelPrices_17_18, plantParameters,  
                                      how='left', 
                                      left_on=['Variable','Year', 'Month'], 
                                      right_on = ['FuelPriceName','Year', 'Month'])

In [195]:
fuelPrices_17_18_plantParameters

,Variable,Date,Price,Year,Month,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost
0,GDA_TETSTX,2018-12-31,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0
1,GDA_TETSTX,2018-12-30,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0
2,GDA_TETSTX,2018-12-29,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0
3,GDA_TETSTX,2018-12-28,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0
4,GDA_TETSTX,2018-12-27,2.915,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Henry Hub,2018-12-27,3.215,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0
1456,Henry Hub,2018-12-28,3.065,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0
1457,Henry Hub,2018-12-29,3.065,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0
1458,Henry Hub,2018-12-30,3.065,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0


In [211]:
fuelPrices_17_18_plantParameters['PowerPriceName'].unique()

array(['HB_SOUTH', nan, 'HB_HOUSTON'], dtype=object)

In [197]:
# RunningCost = ((FuelPrice + FuelTransportationCost) x HeatRate) + VOM

fuelPrices_17_18_plantParameters['RunningCost'] = (
    ((fuelPrices_17_18_plantParameters['Price']+fuelPrices_17_18_plantParameters['FuelTransportationCost'])
     *fuelPrices_17_18_plantParameters['HeatRate'])
     +fuelPrices_17_18_plantParameters['VOM'])

In [198]:
fuelPrices_17_18_plantParameters

,Variable,Date,Price,Year,Month,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost
0,GDA_TETSTX,2018-12-31,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,25.788
1,GDA_TETSTX,2018-12-30,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,25.788
2,GDA_TETSTX,2018-12-29,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,25.788
3,GDA_TETSTX,2018-12-28,3.080,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,25.788
4,GDA_TETSTX,2018-12-27,2.915,2018,12,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,24.534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Henry Hub,2018-12-27,3.215,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,33.498
1456,Henry Hub,2018-12-28,3.065,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,32.118
1457,Henry Hub,2018-12-29,3.065,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,32.118
1458,Henry Hub,2018-12-30,3.065,2018,12,Alpha,HB_HOUSTON,Henry Hub,100.0,9.2,3.0,0.10,10000.0,32.118


# Task 11

In [216]:
powerPrices

,Date,SettlementPoint,Price
0,2016-01-01 00:00:00,HB_BUSAVG,18.42
1,2016-01-01 00:00:00,HB_HOUSTON,18.42
2,2016-01-01 00:00:00,HB_HUBAVG,18.42
3,2016-01-01 00:00:00,HB_NORTH,18.43
4,2016-01-01 00:00:00,HB_SOUTH,18.38
...,...,...,...
129101,2019-12-31 23:00:00,LZ_LCRA,14.53
129102,2019-12-31 23:00:00,LZ_NORTH,14.48
129103,2019-12-31 23:00:00,LZ_RAYBN,14.48
129104,2019-12-31 23:00:00,LZ_SOUTH,14.70


In [219]:
powerPrices['Year'] = powerPrices['Y-M'].apply(lambda x: x.split('/')[1])
powerPrices['Month'] = powerPrices['Y-M'].apply(lambda x: x.split('/')[0])

In [220]:
powerPrices

,Date,SettlementPoint,Price,Y-M,Year,Month
0,2016-01-01 00:00:00,HB_BUSAVG,18.42,01/2016,2016,01
1,2016-01-01 00:00:00,HB_HOUSTON,18.42,01/2016,2016,01
2,2016-01-01 00:00:00,HB_HUBAVG,18.42,01/2016,2016,01
3,2016-01-01 00:00:00,HB_NORTH,18.43,01/2016,2016,01
4,2016-01-01 00:00:00,HB_SOUTH,18.38,01/2016,2016,01
...,...,...,...,...,...,...
129101,2019-12-31 23:00:00,LZ_LCRA,14.53,12/2019,2019,12
129102,2019-12-31 23:00:00,LZ_NORTH,14.48,12/2019,2019,12
129103,2019-12-31 23:00:00,LZ_RAYBN,14.48,12/2019,2019,12
129104,2019-12-31 23:00:00,LZ_SOUTH,14.70,12/2019,2019,12


In [221]:
powerPrices['Year']= powerPrices['Year'].astype(str)
powerPrices['Month']= powerPrices['Month'].astype(str)

In [222]:
powerPrices_17_18 = powerPrices[(powerPrices['Year'] =='2017') | (powerPrices['Year'] == '2018')]

In [223]:
powerPrices_17_18

,Date,SettlementPoint,Price,Y-M,Year,Month
0,2017-01-01 00:00:00,HB_BUSAVG,23.23,01/2017,2017,01
1,2017-01-01 00:00:00,HB_HOUSTON,23.23,01/2017,2017,01
2,2017-01-01 00:00:00,HB_HUBAVG,23.23,01/2017,2017,01
3,2017-01-01 00:00:00,HB_NORTH,23.23,01/2017,2017,01
4,2017-01-01 00:00:00,HB_SOUTH,23.23,01/2017,2017,01
...,...,...,...,...,...,...
122621,2018-12-31 23:00:00,LZ_LCRA,20.54,12/2018,2018,12
122622,2018-12-31 23:00:00,LZ_NORTH,20.49,12/2018,2018,12
122623,2018-12-31 23:00:00,LZ_RAYBN,20.49,12/2018,2018,12
122624,2018-12-31 23:00:00,LZ_SOUTH,21.68,12/2018,2018,12


In [224]:
powerPrices_17_18_HB = powerPrices_17_18[(powerPrices_17_18['SettlementPoint'] =='HB_HOUSTON') | (powerPrices_17_18['SettlementPoint'] == 'HB_SOUTH')]


In [228]:
powerPrices_17_18_HB

,Date,SettlementPoint,Price,Y-M,Year,Month
1,2017-01-01 00:00:00,HB_HOUSTON,23.23,01/2017,2017,01
4,2017-01-01 00:00:00,HB_SOUTH,23.23,01/2017,2017,01
15,2017-01-01 01:00:00,HB_HOUSTON,21.82,01/2017,2017,01
18,2017-01-01 01:00:00,HB_SOUTH,21.82,01/2017,2017,01
29,2017-01-01 02:00:00,HB_HOUSTON,20.16,01/2017,2017,01
...,...,...,...,...,...,...
122588,2018-12-31 21:00:00,HB_SOUTH,24.09,12/2018,2018,12
122599,2018-12-31 22:00:00,HB_HOUSTON,22.03,12/2018,2018,12
122602,2018-12-31 22:00:00,HB_SOUTH,22.67,12/2018,2018,12
122613,2018-12-31 23:00:00,HB_HOUSTON,20.00,12/2018,2018,12


In [231]:
# fuelPrices_17_18_plantParameters

fuelPrices_17_18_plantParameters['Year']= fuelPrices_17_18_plantParameters['Year'].astype(str)
fuelPrices_17_18_plantParameters['Month']= fuelPrices_17_18_plantParameters['Month'].astype(str)

In [232]:
result_df = pd.merge(powerPrices_17_18_HB, fuelPrices_17_18_plantParameters,  
                                      how='left', 
                                      left_on=['SettlementPoint','Year', 'Month'], 
                                      right_on = ['PowerPriceName','Year', 'Month'])

In [234]:
result_df.dropna()

,Date_x,SettlementPoint,Price_x,Y-M,Year,Month,Variable,Date_y,Price_y,PlantName,PowerPriceName,FuelPriceName,Capacity,HeatRate,VOM,FuelTransportationCost,FixedStartCost,RunningCost
13102,2017-10-01 00:00:00,HB_HOUSTON,20.48,10/2017,2017,10,Henry Hub,2017-10-01,2.900,Alpha,HB_HOUSTON,Henry Hub,96.0,9.6,3.0,0.10,10000.0,31.800
13103,2017-10-01 00:00:00,HB_HOUSTON,20.48,10/2017,2017,10,Henry Hub,2017-10-02,2.900,Alpha,HB_HOUSTON,Henry Hub,96.0,9.6,3.0,0.10,10000.0,31.800
13104,2017-10-01 00:00:00,HB_HOUSTON,20.48,10/2017,2017,10,Henry Hub,2017-10-03,2.805,Alpha,HB_HOUSTON,Henry Hub,96.0,9.6,3.0,0.10,10000.0,30.888
13105,2017-10-01 00:00:00,HB_HOUSTON,20.48,10/2017,2017,10,Henry Hub,2017-10-04,2.725,Alpha,HB_HOUSTON,Henry Hub,96.0,9.6,3.0,0.10,10000.0,30.120
13106,2017-10-01 00:00:00,HB_HOUSTON,20.48,10/2017,2017,10,Henry Hub,2017-10-05,2.815,Alpha,HB_HOUSTON,Henry Hub,96.0,9.6,3.0,0.10,10000.0,30.984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297111,2018-12-31 23:00:00,HB_SOUTH,20.67,12/2018,2018,12,GDA_TETSTX,2018-12-05,4.435,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,36.086
297112,2018-12-31 23:00:00,HB_SOUTH,20.67,12/2018,2018,12,GDA_TETSTX,2018-12-04,4.370,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,35.592
297113,2018-12-31 23:00:00,HB_SOUTH,20.67,12/2018,2018,12,GDA_TETSTX,2018-12-03,4.375,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,35.630
297114,2018-12-31 23:00:00,HB_SOUTH,20.67,12/2018,2018,12,GDA_TETSTX,2018-12-02,4.375,Bravo,HB_SOUTH,GDA_TETSTX,250.0,7.6,2.0,0.05,50000.0,35.630


In [ ]:
## running out of time, I can't chech why the merge left generate more than expected!!!